1. 문서의 내용을 읽는다.
2. 문서를 쪼갠다.
   - 토큰수 초과로 답변을 생성하지 못할 수 있고
   - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달


# 1. 문서의 내용을 읽는다.


In [ ]:
!uv add python-docx

In [ ]:
from docx import Document

document = Document("./tax.docx")
print(f"document == {dir(document)}")
full_text = ""
for index, paragraph in enumerate(document.paragraphs):
    print(f"paragraph == {paragraph.text}")
    full_text += f"{paragraph.text}\n"
    # if index == 2:
    #     break

# 2. 문서를 쪼갠다


In [ ]:
!uv add tiktoken

In [ ]:
import tiktoken


def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []

    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i : i + chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    return text_list

In [ ]:
chunk_list = split_text(full_text, 1500)

# 3. 문서 임베딩


In [ ]:
!uv add chromadb

In [ ]:
import chromadb

chroma_client = chromadb.Client()

In [ ]:
collection_name = "tax_collection"
tax_collection = chroma_client.create_collection(collection_name)

In [26]:
import os
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_embedding = OpenAIEmbeddingFunction(
    api_key=openai_api_key, model_name="text-embedding-3-large"
)


In [27]:
mbedding = OpenAIEmbeddingFunction(
    api_key=openai_api_key, model_name="text-embedding-3-large"
)


In [28]:
chroma_client.delete_collection(collection_name)
tax_collection = chroma_client.get_or_create_collection(
    collection_name,
    embedding_function=openai_embedding,
)

In [ ]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f"{index}")

In [32]:
len(id_list)


102

In [31]:
len(chunk_list)

102

In [33]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4. 유사도 검색


In [41]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_doc = tax_collection.query(query_texts=query, n_results=3)

In [42]:
assert retrieved_doc["documents"] is not None
retrieved_doc["documents"][0]

[' 2. 1., 2013. 1. 1., 2013. 3. 22., 2014. 1. 1., 2014. 3. 18., 2014. 12. 23., 2015. 12. 15., 2016. 12. 20., 2018. 3. 20., 2018. 12. 31., 2019. 12. 10., 2019. 12. 31., 2020. 6. 9., 2020. 12. 29., 2022. 8. 12., 2022. 12. 31., 2023. 8. 8., 2023. 12. 31., 2024. 12. 31.>\n1. 「공익신탁법」에 따른 공익신탁의 이익\n2. 사업소득 중 다음 각 목의 어느 하나에 해당하는 소득\n가. 논ㆍ밭을 작물 생산에 이용하게 함으로써 발생하는 소득\n나. 1개의 주택을 소유하는 자의 주택임대소득(제99조에 따른 기준시가가 12억원을 초과하는 주택 및 국외에 소재하는 주택의 임대소득은 제외한다) 또는 해당 과세기간에 대통령령으로 정하는 총수입금액의 합계액이 2천만원 이하인 자의 주택임대소득(2018년 12월 31일 이전에 끝나는 과세기간까지 발생하는 소득으로 한정한다). 이 경우 주택 수의 계산 및 주택임대소득의 산정 등 필요한 사항은 대통령령으로 정한다.\n다. 대통령령으로 정하는 농어가부업소득\n라. 대통령령으로 정하는 전통주의 제조에서 발생하는 소득\n마. 조림기간 5년 이상인 임지(林地)의 임목(林木)의 벌채 또는 양도로 발생하는 소득으로서 연 600만원 이하의 금액. 이 경우 조림기간 및 세액의 계산 등 필요한 사항은 대통령령으로 정한다.\n바. 대통령령으로 정하는 작물재배업에서 발생하는 소득\n사. 대통령령으로 정하는 어로어업 또는 양식어업에서 발생하는 소득\n3. 근로소득과 퇴직소득 중 다음 각 목의 어느 하나에 해당하는 소득\n가. 대통령령으로 정하는 복무 중인 병(兵)이 받는 급여\n나. 법률에 따라 동원된 사람이 그 동원 직장에서 받는 급여\n다. 「산업재해보상보험법」에 따라 수급권자가 받는 요양급여, 휴업급여, 장해급여, 간병급여, 유족급여, 유족특별급

# 5. LLM 질의


In [ ]:
!uv add -q openai

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "system",
            "content": f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요. {retrieved_doc['documents'][0]}",
        },
        {"role": "user", "content": query},
    ],
)


In [46]:
response.choices[0].message.content

'연봉 5천만원인 직장인의 소득세를 계산하려면 크게 세 가지 단계를 거쳐야 합니다:\n\n1. **총급여액에서 근로소득공제**: \n    먼저 연봉에서 근로소득공제를 차감하여 과세표준을 계산해야 합니다. 근로소득공제는 연봉에 따라 다르며 다음과 같습니다:\n\n    - 5천만원 이하: 5천만원의 경우, 근로소득공제는 1,200만원입니다.\n\n2. **과세표준 계산**:\n    - 과세표준 = 총급여액 - 근로소득공제\n    - 과세표준 = 5천만원 - 1,200만원 = 3,800만원\n\n3. **소득세율 적용**:\n    과세표준에 따라 소득세율을 적용합니다. 2023년 현재 적용되는 소득세율은 다음과 같습니다:\n    - 1,200만원 이하: 6%\n    - 1,200만원 초과 ~ 4,600만원 이하: 15%\n    - 4,600만원 초과 ~ 8,800만원 이하: 24%\n    - 8,800만원 초과 ~ 1억5천만원 이하: 35%\n    - 1억5천만원 초과: 38%\n\n    따라서 과세표준 3,800만원에 대한 소득세 계산은 다음과 같습니다:\n    - 1,200만원 이하: 1,200만원 × 6% = 72만원\n    - 1,200만원 초과 ~ 3,800만원까지: (3,800만원 - 1,200만원) × 15% = 2,600만원 × 15% = 390만원\n\n    총 소득세 = 72만원 + 390만원 = 462만원\n\n4. **세액공제**: \n    마지막으로 각종 세액공제를 반영하여 최종 소득세를 산출합니다. 세액공제는 개인마다 다르므로, 정확한 금액은 개인의 공제 항목에 따라 달라집니다.\n\n연봉 5천만원인 경우 소득세를 계산하면 약 462만원이며, 개인의 세액공제를 통해 최종 소득세액은 달라질 수 있음을 참고하시기 바랍니다.'